# Part 4: XML creation
## Dependencies: OTIO and pandas



In [1]:
#audio
from __future__ import print_function
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

import librosa
import librosa.display

In [2]:
import pandas as pd 
import pyloudnorm as pyln

In [3]:
#video
import moviepy
from moviepy.editor import *
import opentimelineio as otio
import argparse

In [4]:
import cv2

### Quick summary, OTIO can output XML that CAN be used to import timelines. We just need to learn the behavior and then create a loop to automate clipping, using a imported dataframe as a outline.

# Timeline Outline behavior testing
- First create a list of videos
    - sfs
- Sec create list of audio

- Link audio to video, IE if mic1 use center camera

## Use the lists of camera and audio to createa loops creating timeranges

In [5]:
VIDEO_FILEPATH_LIST= ["Hang out 2 Middle-004.mp4","Hang out 2 Left-001.mp4","Hang out 2 Right.mp4"]

In [6]:
AUDIO_VIDEO_TUPLE_LIST=[("E2 - Chukwu.mp3", 1), ("E2 - Osi.mp3", 1),("E2 - Crystal.mp3",2),("E2 - Scott.mp3",2)]

In [7]:

# build the structure
tl = otio.schema.Timeline(name="Example timeline")

# add track for each video file and each audio file
#for each file add a track
for i in VIDEO_FILEPATH_LIST :
    tr = otio.schema.Track(name=i)
    tl.tracks.append(tr)

for i in AUDIO_VIDEO_TUPLE_LIST:
    tr = otio.schema.Track(name=i[0])
    tl.tracks.append(tr)

# Further data editing 

## Create a camera view column that matches the audio_video_tuple to  idxmax

In [8]:
data_df=pd.read_pickle('idxmax.pkl')

In [9]:
#use indexes of when crossover is 1 to change cam_view to 0 for three seconds after crossover

In [10]:

cap = cv2. VideoCapture("Middle View.mp4")
vlength = int(cap. get(cv2. CAP_PROP_FRAME_COUNT))
alength=len(data_df)

In [11]:
alength
vlength

0

In [12]:
data_df

,idxmax,crossover,high_mult_thresh,low_mult_thresh,nearnesssound,A0,A1,A2,A3,Main_cam
0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...
1733491,0,0,0,0,0,0,0,0,0,0
1733492,0,0,0,0,0,0,0,0,0,0
1733493,0,0,0,0,0,0,0,0,0,0
1733494,0,0,0,0,0,0,0,0,0,0


In [13]:
#create an empty

#if crossover is 1, change that column +rate*3

## Create intervals of data in tuple form. 
## Audio Tuples represent the start and end frame of when mic is dominant. Camera 0 is default cam in case of uncertainty, which is center cam in this scenario.
## Video Tuples represent the start and end frame of which mic to use. 


In [14]:
def dataframe_getintervals(series,desiredvalue):
    #make sure series is the df['column']
    t=series.index[series==desiredvalue].to_series()
    interval_list=t.groupby(t.diff().ne(1).cumsum()).agg(['first','last']).apply(tuple,1).tolist()
    
    return interval_list

In [15]:
#create lists of tuples for each camera based on the audio

list_of_idxmax_mic_data=[]
for i in range(len(AUDIO_VIDEO_TUPLE_LIST)+1):
    tuple_list=dataframe_getintervals(data_df['idxmax'],i)
    list_of_idxmax_mic_data.append(tuple_list)

    
##create lists of tuples for each Audio

In [16]:
#creating a list to referencing the column names of df 
list_str_audio_thresholds=[]
for irow in range(len(AUDIO_VIDEO_TUPLE_LIST)):
    varname="A"
    list_str_audio_thresholds.append(varname+str(irow))

In [17]:
#create lists of tuples for each audio based the threshold region
list_of_tuples_threshold_mic_data=[]
for i in range(len(AUDIO_VIDEO_TUPLE_LIST)):
    tuple_list=dataframe_getintervals(data_df[list_str_audio_thresholds[i]],1)
    list_of_tuples_threshold_mic_data.append(tuple_list)

In [18]:
xml_to_otio_test=otio.adapters.read_from_file('Fixed Settings.xml')


In [19]:
xml_to_otio_test.metadata

{'fcp_xml': {'@AM.TrackScrollPosition': '0', '@AM.TrackVScrollPosition': '75', '@MZ.EditLine': '6600425832000', '@MZ.Sequence.AudioTimeDisplayFormat': '200', '@MZ.Sequence.EditingModeGUID': '795454d9-d3c2-429d-9474-923ab13b7018', '@MZ.Sequence.PreviewFrameSizeHeight': '1080', '@MZ.Sequence.PreviewFrameSizeWidth': '1920', '@MZ.Sequence.PreviewRenderingClassID': '1061109567', '@MZ.Sequence.PreviewRenderingPresetCodec': '1297107278', '@MZ.Sequence.PreviewRenderingPresetPath': 'EncoderPresets/SequencePreview/795454d9-d3c2-429d-9474-923ab13b7018/I-Frame Only MPEG.epr', '@MZ.Sequence.PreviewUseMaxBitDepth': 'false', '@MZ.Sequence.PreviewUseMaxRenderQuality': 'false', '@MZ.Sequence.VideoTimeDisplayFormat': '110', '@Monitor.ProgramZoomIn': '0', '@Monitor.ProgramZoomOut': '279167288400000', '@TL.SQAVDividerPosition': '0.5', '@TL.SQAudioVisibleBase': '0', '@TL.SQHeaderWidth': '236', '@TL.SQHideShyTracks': '0', '@TL.SQTimePerPixel': '0.19999999999999998', '@TL.SQVideoVisibleBase': '0', '@TL.SQVis

In [20]:
{'fcp_xml': {'@AM.TrackScrollPosition': '0', '@AM.TrackVScrollPosition': '75', '@MZ.EditLine': '6600425832000', '@MZ.Sequence.AudioTimeDisplayFormat': '200', '@MZ.Sequence.EditingModeGUID': '795454d9-d3c2-429d-9474-923ab13b7018', '@MZ.Sequence.PreviewFrameSizeHeight': '1080', '@MZ.Sequence.PreviewFrameSizeWidth': '1920', '@MZ.Sequence.PreviewRenderingClassID': '1061109567', '@MZ.Sequence.PreviewRenderingPresetCodec': '1297107278', '@MZ.Sequence.PreviewRenderingPresetPath': 'EncoderPresets/SequencePreview/795454d9-d3c2-429d-9474-923ab13b7018/I-Frame Only MPEG.epr', '@MZ.Sequence.PreviewUseMaxBitDepth': 'false', '@MZ.Sequence.PreviewUseMaxRenderQuality': 'false', '@MZ.Sequence.VideoTimeDisplayFormat': '110', '@Monitor.ProgramZoomIn': '0', '@Monitor.ProgramZoomOut': '279167288400000', '@TL.SQAVDividerPosition': '0.5', '@TL.SQAudioVisibleBase': '0', '@TL.SQHeaderWidth': '236', '@TL.SQHideShyTracks': '0', '@TL.SQTimePerPixel': '0.19999999999999998', '@TL.SQVideoVisibleBase': '0', '@TL.SQVisibleBaseTime': '0', '@explodedTracks': 'true', '@id': 'sequence-2', 'labels': {'label2': 'Forest'}, 'logginginfo': {'description': None, 'good': None, 'lognote': None, 'originalaudiofilename': None, 'originalvideofilename': None, 'scene': None, 'shottake': None}, 'media': {'audio': {'format': {'samplecharacteristics': {'depth': '16', 'samplerate': '48000'}}, 'numOutputChannels': '2', 'outputs': {'group': [{'channel': {'index': '1'}, 'downmix': '0', 'index': '1', 'numchannels': '1'}, {'channel': {'index': '2'}, 'downmix': '0', 'index': '2', 'numchannels': '1'}]}}, 'video': {'format': {'samplecharacteristics': {'anamorphic': 'FALSE', 'codec': {'appspecificdata': {'appmanufacturer': 'Apple Inc.', 'appname': 'Final Cut Pro', 'appversion': '7.0', 'data': {'qtcodec': {'codecname': 'Apple ProRes 422', 'codectypecode': 'apcn', 'codectypename': 'Apple ProRes 422', 'codecvendorcode': 'appl', 'datarate': '0', 'keyframerate': '0', 'spatialquality': '1024', 'temporalquality': '0'}}}, 'name': 'Apple ProRes 422'}, 'colordepth': '24', 'fielddominance': 'none', 'height': '2160', 'pixelaspectratio': 'square', 'rate': {'ntsc': 'TRUE', 'timebase': '24'}, 'width': '3840'}}}}, 'rate': {'ntsc': 'TRUE', 'timebase': '24'}, 'timecode': {'displayformat': 'NDF', 'rate': {'ntsc': 'TRUE', 'timebase': '24'}}, 'uuid': '377abc11-205d-498b-b058-6e73c151d97f'}}

{'fcp_xml': {'@AM.TrackScrollPosition': '0',
  '@AM.TrackVScrollPosition': '75',
  '@MZ.EditLine': '6600425832000',
  '@MZ.Sequence.AudioTimeDisplayFormat': '200',
  '@MZ.Sequence.EditingModeGUID': '795454d9-d3c2-429d-9474-923ab13b7018',
  '@MZ.Sequence.PreviewFrameSizeHeight': '1080',
  '@MZ.Sequence.PreviewFrameSizeWidth': '1920',
  '@MZ.Sequence.PreviewRenderingClassID': '1061109567',
  '@MZ.Sequence.PreviewRenderingPresetCodec': '1297107278',
  '@MZ.Sequence.PreviewRenderingPresetPath': 'EncoderPresets/SequencePreview/795454d9-d3c2-429d-9474-923ab13b7018/I-Frame Only MPEG.epr',
  '@MZ.Sequence.PreviewUseMaxBitDepth': 'false',
  '@MZ.Sequence.PreviewUseMaxRenderQuality': 'false',
  '@MZ.Sequence.VideoTimeDisplayFormat': '110',
  '@Monitor.ProgramZoomIn': '0',
  '@Monitor.ProgramZoomOut': '279167288400000',
  '@TL.SQAVDividerPosition': '0.5',
  '@TL.SQAudioVisibleBase': '0',
  '@TL.SQHeaderWidth': '236',
  '@TL.SQHideShyTracks': '0',
  '@TL.SQTimePerPixel': '0.19999999999999998',
  '